# Création des cartes de membres

## QR Code

In [1]:
import numpy as np
import pandas as pd
import qrcode
import matplotlib.pyplot as plt
import imageio.v2 as iio
import skimage.transform as skt


def format_name(first_name: str, last_name: str) -> str:
    """
    Format the name of a person
    :param first_name: first name
    :param last_name: last name
    :return: formatted name
    """
    return f'{first_name} {last_name}'


def double_size(img: np.array) -> np.array:
    """
    Double the size of an image using nearest neighbor interpolation.
    :param img: image to double
    :return: doubled image
    """
    return np.kron(img, np.ones((2, 2)))

def resize(img: np.array, size: tuple) -> np.array:
    """
    :param img: image to resize
    :param size: new size of the image (height, width) (int, int)
    :return: resized image
    """
    return skt.resize(img, size, anti_aliasing=True)

def bw_to_rgb(img: np.array, color=(1.0, 1.0, 1.0), rev=False, b_to_w=False) -> np.array:
    """
    2D array to 3D array
    :param img: 2D image
    :param color: color to apply to the white pixels
    :param rev: reverse the image so the color is applied to the black pixels
    :param b_to_w: change black pixels to white (last operation)
    :return: 3D image
    """
    if rev: img = 1.0 - img
    img = np.stack([img*color[0], img*color[1], img*color[2]], axis=2)
    if b_to_w: 
        mask = np.all(img == [0.0, 0.0, 0.0], axis=2)
        img[mask] = [1.0, 1.0, 1.0]
    return img

def generate_qr_code(data: str) -> np.array:
    """
    Generate a QR code from a string
    :param data: string to encode
    :return: QR code image
    """
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=10,
        border=0,
    )
    qr.add_data(data)
    qr.make(fit=True)
    return np.array(qr.make_image(fill_color="black", back_color="white"))

def member_to_qr_code_str(member: pd.Series) -> str:
    """
    Creates a json string from a member
    :param member: member to encode
    - member_id
    - expiration_date
    - first_name
    - last_name
    - hash
    :return: json string
    """
    user = {}
    user['CCUdeM'] = member['member_id']
    user['exp'] = member['expiration_date']
    user['nom'] = format_name(member['first_name'], member['last_name'])
    user['h'] = member['hash']
    return str(user)

def append_rgba_to_rgb(rgb: np.array, rgba: np.array) -> np.array:
    """
    Append an RGBA image to an RGB image using the alpha channel
    Both images must have the same size
    :param rgb: RGB image
    :param rgba: RGBA image to append on top of the RGB image
    :return: RGB image with the RGBA image on top
    """
    alpha = rgba[:, :, 3]
    alpha = np.stack([alpha, alpha, alpha], axis=2)
    rgb = rgb * (1 - alpha) + rgba[:, :, :3] * alpha
    return rgb

def add_padding(bw: np.array, new_size: tuple, value) -> np.array:
    """
    Add padding a 2D image
    :param bw: 2D image
    :param new_size: new size of the image (height, width) (int, int)
    :param value: value to fill the padding
    :return: padded image
    """
    pad_x = (new_size[0] - bw.shape[0]) // 2
    pad_y = (new_size[1] - bw.shape[1]) // 2
    pad = ((pad_x, pad_x), (pad_y, pad_y))
    return np.pad(bw, pad, mode='constant', constant_values=value)

def add_padding_rgba(rgba: np.array, new_size: tuple) -> np.array:
    """
    Add padding to an RGBA image
    The value of the padding is 1.0 for the RGB channels and 0.0 for the alpha channel
    :param rgba: RGBA image
    :param new_size: new size of the image (height, width) (int, int)
    :return: padded RGBA image
    """
    r = add_padding(rgba[:, :, 0], new_size, 1.0)
    g = add_padding(rgba[:, :, 1], new_size, 1.0)
    b = add_padding(rgba[:, :, 2], new_size, 1.0)
    a = add_padding(rgba[:, :, 3], new_size, 0.0)
    return np.stack([r, g, b, a], axis=2)

def scale_mask(rgba: np.array, scale) -> np.array:
    """
    Scale the alpha channel of an RGBA image
    :param rgba: RGBA image
    :param scale: scale factor
    :return: same image with the scaled alpha channel
    """
    mask = rgba[:, :, 3]
    original_shape = mask.shape
    size = (int(mask.shape[0] * scale), int(mask.shape[1] * scale))
    mask = skt.resize(mask, size, anti_aliasing=True)
    new_shape = mask.shape
    x = (new_shape[0] - original_shape[0]) // 2
    y = (new_shape[1] - original_shape[1]) // 2
    mask = mask[x:x + original_shape[0], y:y + original_shape[1]]
    rgba[:, :, 3] = mask
    return rgba

def add_logo(qr: np.array, logo: np.array, qr_color=tuple) -> np.array:
    """
    Add a logo to a QR code
    :param qr: QR code image
    :param logo: logo image
    :param qr_color: color of the QR code (R, G, B) (0-255)
    :return: QR code with the logo
    """
    color = np.array(qr_color)/255.0
    qr = double_size(qr)
    qr = bw_to_rgb(qr, color, rev=True, b_to_w=True)
    qr = resize(qr, (2550, 2550))
    logo = resize(logo, (600, 600))
    logo = add_padding_rgba(logo, qr.shape)
    logo = scale_mask(logo, 1.1)
    qr = append_rgba_to_rgb(qr, logo)
    return qr


def ccudem_qr_code(member: pd.Series, logo: np.array) -> np.array:
    member = member_to_qr_code_str(member)
    qr = generate_qr_code(member)
    qr = add_logo(qr, logo, (0, 89, 158))
    qr = (qr * 255).astype(np.uint8)
    return qr



## Carte membre

In [2]:
from PIL import Image, ImageDraw, ImageFont


def add_text_to_array(array, y, x, text, text_size=50, color=(0,0,0), font_path='./fonts/BAHNSCHRIFT.TTF'):
    pil_image = Image.fromarray(array)
    font = ImageFont.truetype(font_path, size=text_size)
    draw = ImageDraw.Draw(pil_image)
    text_bbox = draw.textbbox((x, y), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    text_x = x - text_width // 2
    text_y = y - text_height // 2
    draw.text((text_x, text_y), text, fill=color, font=font)
    modified_array = np.array(pil_image)
    return modified_array

def get_empty_card(dpi=1200):
    card_size = np.array([85.6, 53.98])
    card_size = card_size / 25.4 * dpi
    card_size = card_size.astype(int)
    card = np.ones((card_size[0], card_size[1], 3), dtype=np.uint8) * 255
    return card

def crop_white(array: np.array) -> np.array:
    i_min = np.min(np.where(array != 255)[0])
    j_min = np.min(np.where(array != 255)[1])
    i_max = np.max(np.where(array != 255)[0])
    j_max = np.max(np.where(array != 255)[1])
    return array[i_min:i_max, j_min:j_max]

def remove_pipe(array: np.array) -> np.array:
    """
    :param array: 3D array
    :return: 3D array without the pipe character in the image
    """
    line = array[array.shape[0] // 2, :, 0]
    pipe_width = np.min(np.where(line == 255))
    array = array[:, pipe_width:]
    white_width = np.min(np.where(array != 255)[1])
    array = array[:, white_width:]
    return array

def get_text_box(text: str, text_size: int, color: tuple, font_path: str, size=3840) -> np.array:
    text = '|' + text
    array = np.ones((size, size, 3), dtype=np.uint8) * 255
    array = add_text_to_array(array, int(size//2), int(size//2), text, text_size, color, font_path)
    array = crop_white(array)
    array = remove_pipe(array)
    return array

def format_number(num: int) -> str:
    num = f"{num:06}"
    num = ' '.join([num[i:i+3] for i in range(0, len(num), 3)])
    return num

def format_date(date: str) -> str:
    date = pd.Timestamp(date)
    return date.strftime('%Y.%m')

def overlay_bottom_left(array: np.array, overlay: np.array, y: int, x: int) -> np.array:
    shape = overlay.shape
    array[y-shape[0]:y, x:x+shape[1]] = overlay
    return array

def overlay_bottom_right(array: np.array, overlay: np.array, y: int, x: int) -> np.array:
    shape = overlay.shape
    array[y-shape[0]:y, x-shape[1]:x] = overlay
    return array

def overlay_top_left(array: np.array, overlay: np.array, y: int, x: int) -> np.array:
    shape = overlay.shape
    array[y:y+shape[0], x:x+shape[1]] = overlay
    return array

def overlay_top_right(array: np.array, overlay: np.array, y: int, x: int) -> np.array:
    shape = overlay.shape
    array[y:y+shape[0], x-shape[1]:x] = overlay
    return array



In [3]:
def add_member_to_card(member: pd.Series, logo: np.array) -> np.array:
    padding = 150
    blue = (0, 89, 158)
    black = (0, 0, 0)
    white = (255, 255, 255)

    light = 'fonts/Figtree/static/Figtree-Light.ttf'
    semi_bold = 'fonts/Figtree/static/Figtree-SemiBold.ttf'
    bold = 'fonts/Figtree/static/Figtree-Bold.ttf'

    card = get_empty_card(dpi=1200)
    qr = ccudem_qr_code(member, logo)
    size = card.shape[1] - (2 * padding)
    qr = (resize(qr, (size, size, 3)) * 255).astype(np.uint8)

    card[-size-padding-100:-padding-100, padding:padding+size] = qr
    card[padding:500, padding:-padding] = np.array(blue)

    text = 'CLUB CYCLISME UNIVERSITÉ DE MONTRÉAL'
    card = add_text_to_array(
        card, 200, card.shape[1] // 2, 
        text_size=108,
        color=white,
        text=text,
        font_path=light
    )

    # MEMBRE

    name = member['first_name'] + ' ' + member['last_name']
    name_img = get_text_box(name, 200, black, light)
    if name_img.shape[1] > card.shape[1] - (2 * padding):
        name_img = (resize(name_img, (name_img.shape[0], card.shape[1] - (2 * padding))) * 255).astype(np.uint8)
    card = overlay_bottom_left(card, name_img, 1500, padding)

    type = member['type'].upper()
    type_img = get_text_box(type, 80, blue, light)
    card = overlay_bottom_left(card, type_img, 1275, padding)

    # NUMERO

    numero = f'{format_number(member["member_id"])}'
    num_img = get_text_box(numero, 80, black, semi_bold)
    card = overlay_top_right(card, num_img, 3825, card.shape[1]-padding)

    mem_no = 'MEMBRE NUMERO :'
    mem_no_img = get_text_box(mem_no, 80, black, light)
    card = overlay_top_right(card, mem_no_img, 3825, card.shape[1]-padding-350)

    # INSCRIPTION

    join_date = format_date(member['join_date'])
    join_date_img = get_text_box(join_date, 280, black, bold)
    card = overlay_top_left(card, join_date_img, 550, padding)

    inscription = 'PREMIÈRE INSCRIPTION'
    inscription_img = get_text_box(inscription, 80, black, light)
    card = overlay_top_left(card, inscription_img, 800, padding)

    # EXPIRATION

    expiration_date = format_date(member['expiration_date'])
    expiration_date_img = get_text_box(expiration_date, 280, blue, bold)
    card = overlay_top_right(card, expiration_date_img, 550, -padding)

    expiration = 'EXPIRATION'
    expiration_img = get_text_box(expiration, 80, black, light)
    card = overlay_top_right(card, expiration_img, 800, -padding)

    return card

In [4]:
# img = iio.imread('ccudem/cyclisme_sans-degrade.png')
img = iio.imread('https://github.com/comtois-etienne/ccudem/blob/main/ccudem/cyclisme_sans-degrade.png?raw=true')
members = pd.read_csv('members.csv')

for _, member in members.iterrows():
    # todo : if member is active
    first_name = f'qr_codes/{member["first_name"]}{member["last_name"]}.png'
    card = add_member_to_card(member, img)
    print(first_name)
    # plt.imshow(card)
    # fig = plt.gcf()
    # fig.set_size_inches(20, 10)
    # plt.show()
    iio.imwrite(first_name, card)

qr_codes/EtienneComtois.png
qr_codes/PhilippeDessureault.png
